In [1]:
from datasets import load_dataset, IterableDataset,Dataset
import pandas as pd



C:\Users\harsh\anaconda3\envs\nlp_hw1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Reading fire incidence data into a dataframe
df=pd.read_csv('Fire_Incident_Dispatch_Data_Current.csv')

#Reading file containing modified zipcodes
modified_zipcodes=pd.read_excel("Modified and original zipcodes.xlsx")

#Merging Fire incidence data with Modified Ziocodes information
Merged=pd.merge(df,modified_zipcodes, how='left',left_on='ZIPCODE', right_on='ZCTA')

In [3]:
#Reading file containing information about tabulation areas

dataset_zip=pd.read_csv('Modified_Zip_Code_Tabulation_Areas__MODZCTA__20250323.csv')


#Determing the centroids
from shapely.geometry import MultiPolygon, Polygon
from shapely.wkt import loads


def centroidfun(x):
    multipol=loads(x)
    centroid=multipol.centroid
    return centroid

dataset_zip["centroid"]=dataset_zip["the_geom"].apply(centroidfun)




In [4]:
#Extracting latitude and longitude
dataset_zip['Latitude']=dataset_zip['centroid'].apply(lambda point:point.x if point is not None else None)
dataset_zip['Longitude']=dataset_zip['centroid'].apply(lambda point:point.y if point is not None else None)

In [5]:
print(Merged.columns)
print(dataset_zip.columns)
dataset_zip.head()



Index(['STARFIRE_INCIDENT_ID', 'INCIDENT_DATETIME', 'ALARM_BOX_BOROUGH',
       'ALARM_BOX_NUMBER', 'ALARM_BOX_LOCATION', 'INCIDENT_BOROUGH', 'ZIPCODE',
       'POLICEPRECINCT', 'CITYCOUNCILDISTRICT', 'COMMUNITYDISTRICT',
       'COMMUNITYSCHOOLDISTRICT', 'CONGRESSIONALDISTRICT',
       'ALARM_SOURCE_DESCRIPTION_TX', 'ALARM_LEVEL_INDEX_DESCRIPTION',
       'HIGHEST_ALARM_LEVEL', 'INCIDENT_CLASSIFICATION',
       'INCIDENT_CLASSIFICATION_GROUP', 'DISPATCH_RESPONSE_SECONDS_QY',
       'FIRST_ASSIGNMENT_DATETIME', 'FIRST_ACTIVATION_DATETIME',
       'FIRST_ON_SCENE_DATETIME', 'INCIDENT_CLOSE_DATETIME',
       'VALID_DISPATCH_RSPNS_TIME_INDC', 'VALID_INCIDENT_RSPNS_TIME_INDC',
       'INCIDENT_RESPONSE_SECONDS_QY', 'INCIDENT_TRAVEL_TM_SECONDS_QY',
       'ENGINES_ASSIGNED_QUANTITY', 'LADDERS_ASSIGNED_QUANTITY',
       'OTHER_UNITS_ASSIGNED_QUANTITY', 'ZCTA', 'MODZCTA'],
      dtype='object')
Index(['MODZCTA', 'label', 'ZCTA', 'pop_est', 'the_geom', 'centroid',
       'Latitude', 'Longitude

,MODZCTA,label,ZCTA,pop_est,the_geom,centroid,Latitude,Longitude
0,10001,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...,POINT (-73.99713787189704 40.75068819675726),-73.997138,40.750688
1,10002,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...,POINT (-73.98617431136368 40.71578031863019),-73.986174,40.715780
2,10003,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...,POINT (-73.9891594531684 40.73182645700668),-73.989159,40.731826
3,10026,10026,10026,39363,MULTIPOLYGON (((-73.96200720826798 40.80550908...,POINT (-73.95267194513652 40.802382116396615),-73.952672,40.802382
4,10004,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...,POINT (-74.01303190579041 40.70374683920746),-74.013032,40.703747


In [6]:
dataset_zip['centroid']

0       POINT (-73.99713787189704 40.75068819675726)
1       POINT (-73.98617431136368 40.71578031863019)
2        POINT (-73.9891594531684 40.73182645700668)
3      POINT (-73.95267194513652 40.802382116396615)
4       POINT (-74.01303190579041 40.70374683920746)
                           ...                      
173     POINT (-73.78338459367053 40.65686444670272)
174     POINT (-73.80961947850157 40.70128901923497)
175     POINT (-73.84455125213255 40.57759893303544)
176     POINT (-73.90719766100317 40.55906730906841)
177    POINT (-73.93966853865346 40.709534006494096)
Name: centroid, Length: 178, dtype: object

In [7]:
#Rows containing Nan's in the MODZCTA
merged_nan=Merged[Merged['MODZCTA'].isna()]
print(len(merged_nan))

5808


In [8]:
#Rows not containing Nan's in the MODZCTA
fire_incidence=Merged[Merged['MODZCTA'].notna()]
fire_incidence['MODZCTA']=fire_incidence['MODZCTA'].astype(int)

print(len(fire_incidence))

1221599


C:\Users\harsh\AppData\Local\Temp\ipykernel_18212\1421915464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_incidence['MODZCTA']=fire_incidence['MODZCTA'].astype(int)


In [9]:
#Grouping the dataset by MODZCTA

fire_grouped=fire_incidence.groupby('MODZCTA').size().reset_index(name='Count')




In [10]:
#Adding the latitude and longitude values
Merged_latlong=pd.merge(fire_grouped,dataset_zip, how='left',left_on='MODZCTA', right_on='MODZCTA')

In [11]:
Merged_latlong

,MODZCTA,Count,label,ZCTA,pop_est,the_geom,centroid,Latitude,Longitude
0,10001,10189,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...,POINT (-73.99713787189704 40.75068819675726),-73.997138,40.750688
1,10002,16990,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...,POINT (-73.98617431136368 40.71578031863019),-73.986174,40.715780
2,10003,10210,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...,POINT (-73.9891594531684 40.73182645700668),-73.989159,40.731826
3,10004,1419,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...,POINT (-74.01303190579041 40.70374683920746),-74.013032,40.703747
4,10005,1434,10005,"10005, 10271",8831,MULTIPOLYGON (((-74.00782636323468 40.70308666...,POINT (-74.00887017653658 40.706100057005145),-74.008870,40.706100
...,...,...,...,...,...,...,...,...,...
172,11691,9990,11691,11691,67094,MULTIPOLYGON (((-73.78915889414289 40.59752681...,POINT (-73.76135910415132 40.60132941338052),-73.761359,40.601329
173,11692,2729,11692,11692,20991,MULTIPOLYGON (((-73.80276329921944 40.59261580...,POINT (-73.79219583977951 40.5927973858467),-73.792196,40.592797
174,11693,2129,11693,11693,12919,MULTIPOLYGON (((-73.82010481980066 40.58699545...,POINT (-73.81671090698718 40.596457309804165),-73.816711,40.596457
175,11694,3136,11694,11694,21354,MULTIPOLYGON (((-73.86495602260939 40.56663006...,POINT (-73.84455125213255 40.57759893303544),-73.844551,40.577599


In [12]:
#Classification of the fires into three different risk levels

highrisk=['Automobile Fire','Brush Fire','Carbon Monoxide - Code 3 - Emergency (over 9 ppm)','Church Fire','Factory Fire','Hospital Fire','Manhole Fire - Extended to Building','Maritime Fire','Medical - Serious Life Threatening',"Multiple Dwelling 'B' Fire",'Other Commercial Building Fire','Other Public Building Fire','Private Dwelling Fire','School Fire','Store Fire','Theater or TV Studio Fire','Transit System - Structural','Under Construction / Vacant Fire','Vehicle Accident - With Extrication']
mediumrisk=['Abandoned Derelict Vehicle Fire','Carbon Monoxide - Code 2 - Incident (1-9 ppm)','Defective Oil Burner','Demolition Debris or Rubbish Fire','Elevator Emergency - Occupied','Manhole Fire - Blown Cover','Manhole Fire - Other','Manhole Fire - Seeping Smoke','Maritime Emergency','Medical - Breathing / Ill or Sick',"Multiple Dwelling 'A' - Compactor fire","Multiple Dwelling 'A' - Food on the stove fire","Multiple Dwelling 'A' - Other fire",'Other Transportation Fire','Sprinkler System - Activated','Transit System - NonStructural','Transit System Emergency','Utility Emergency - Electric','Utility Emergency - Gas','Utility Emergency - Steam','Vehicle Accident - Other']
lowrisk=['Alarm System - Defective','Alarm System - Testing','Alarm System - Unnecessary','Assist Civilian - Non-Medical','Carbon Monoxide - Code 1 - Investigation','Carbon Monoxide - Code 4 - No Detector Activation','Downed Tree','Elevator Emergency - Unoccupied','Medical - Assist Civilian','Medical - EMS Link 10-91','Medical - No PT Contact EMS is Onscene','Medical - PD Link 10-91','Medical - Victim Deceased','Medical MFA - EMS Link','Medical MFA - PD Link','Non-Medical 10-91 (Unnecessary Alarm)','Non-Medical MFA - BARS','Non-Medical MFA - ERS','Non-Medical MFA - Phone','Non-Medical MFA - Private Fire Alarm','Non-Medical MFA - Verbal','Odor - Other Smoke','Odor - Other Than Smoke','Remove Civilian - Non-Fire','Sprinkler System - Malfunction','Sprinkler System - Working on System','Undefined Emergency','Utility Emergency - Undefined','Utility Emergency - Water']

#Generating three dataframes

highrisk_df=Merged[Merged['INCIDENT_CLASSIFICATION'].isin(highrisk)]
mediumrisk_df=Merged[Merged['INCIDENT_CLASSIFICATION'].isin(mediumrisk)]
lowrisk_df=Merged[Merged['INCIDENT_CLASSIFICATION'].isin(lowrisk)]

#Grouping the above three datasets based on Zipcode

highrisk_df_grouped=highrisk_df.groupby('MODZCTA').size().reset_index(name='Count')
mediumrisk_df_grouped=mediumrisk_df.groupby('MODZCTA').size().reset_index(name='Count')
lowrisk_df_grouped=lowrisk_df.groupby('MODZCTA').size().reset_index(name='Count')




In [13]:
#Adding the latitude and longitude values
highrisk_csv=pd.merge(highrisk_df_grouped,dataset_zip, how='left',left_on='MODZCTA', right_on='MODZCTA')
mediumrisk_csv=pd.merge(mediumrisk_df_grouped,dataset_zip, how='left',left_on='MODZCTA', right_on='MODZCTA')
lowrisk_csv=pd.merge(lowrisk_df_grouped,dataset_zip, how='left',left_on='MODZCTA', right_on='MODZCTA')

In [43]:
#Exporting data into csv files

highrisk_csv.to_csv('HighRiskData.csv', index=False)
mediumrisk_csv.to_csv('MediumRiskData.csv', index=False)
lowrisk_csv.to_csv('LowRiskData.csv', index=False)
Merged_latlong.to_csv('FireIncidenceAll.csv',index=False)

#


In [37]:
highrisk_csv

,MODZCTA,Count,label,ZCTA,pop_est,the_geom,centroid,Latitude,Longitude
0,10001.0,352,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...,POINT (-73.99713787189704 40.75068819675726),-73.997138,40.750688
1,10002.0,516,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...,POINT (-73.98617431136368 40.71578031863019),-73.986174,40.715780
2,10003.0,341,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...,POINT (-73.9891594531684 40.73182645700668),-73.989159,40.731826
3,10004.0,47,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...,POINT (-74.01303190579041 40.70374683920746),-74.013032,40.703747
4,10005.0,52,10005,"10005, 10271",8831,MULTIPOLYGON (((-74.00782636323468 40.70308666...,POINT (-74.00887017653658 40.706100057005145),-74.008870,40.706100
...,...,...,...,...,...,...,...,...,...
172,11691.0,768,11691,11691,67094,MULTIPOLYGON (((-73.78915889414289 40.59752681...,POINT (-73.76135910415132 40.60132941338052),-73.761359,40.601329
173,11692.0,178,11692,11692,20991,MULTIPOLYGON (((-73.80276329921944 40.59261580...,POINT (-73.79219583977951 40.5927973858467),-73.792196,40.592797
174,11693.0,128,11693,11693,12919,MULTIPOLYGON (((-73.82010481980066 40.58699545...,POINT (-73.81671090698718 40.596457309804165),-73.816711,40.596457
175,11694.0,219,11694,11694,21354,MULTIPOLYGON (((-73.86495602260939 40.56663006...,POINT (-73.84455125213255 40.57759893303544),-73.844551,40.577599


In [39]:
mediumrisk_csv

,MODZCTA,Count,label,ZCTA,pop_est,the_geom,centroid,Latitude,Longitude
0,10001.0,3135,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...,POINT (-73.99713787189704 40.75068819675726),-73.997138,40.750688
1,10002.0,7091,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...,POINT (-73.98617431136368 40.71578031863019),-73.986174,40.715780
2,10003.0,3215,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...,POINT (-73.9891594531684 40.73182645700668),-73.989159,40.731826
3,10004.0,420,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...,POINT (-74.01303190579041 40.70374683920746),-74.013032,40.703747
4,10005.0,418,10005,"10005, 10271",8831,MULTIPOLYGON (((-74.00782636323468 40.70308666...,POINT (-74.00887017653658 40.706100057005145),-74.008870,40.706100
...,...,...,...,...,...,...,...,...,...
172,11691.0,3268,11691,11691,67094,MULTIPOLYGON (((-73.78915889414289 40.59752681...,POINT (-73.76135910415132 40.60132941338052),-73.761359,40.601329
173,11692.0,879,11692,11692,20991,MULTIPOLYGON (((-73.80276329921944 40.59261580...,POINT (-73.79219583977951 40.5927973858467),-73.792196,40.592797
174,11693.0,629,11693,11693,12919,MULTIPOLYGON (((-73.82010481980066 40.58699545...,POINT (-73.81671090698718 40.596457309804165),-73.816711,40.596457
175,11694.0,1119,11694,11694,21354,MULTIPOLYGON (((-73.86495602260939 40.56663006...,POINT (-73.84455125213255 40.57759893303544),-73.844551,40.577599


In [41]:
lowrisk_csv

,MODZCTA,Count,label,ZCTA,pop_est,the_geom,centroid,Latitude,Longitude
0,10001.0,6702,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...,POINT (-73.99713787189704 40.75068819675726),-73.997138,40.750688
1,10002.0,9382,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...,POINT (-73.98617431136368 40.71578031863019),-73.986174,40.715780
2,10003.0,6653,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...,POINT (-73.9891594531684 40.73182645700668),-73.989159,40.731826
3,10004.0,952,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...,POINT (-74.01303190579041 40.70374683920746),-74.013032,40.703747
4,10005.0,963,10005,"10005, 10271",8831,MULTIPOLYGON (((-74.00782636323468 40.70308666...,POINT (-74.00887017653658 40.706100057005145),-74.008870,40.706100
...,...,...,...,...,...,...,...,...,...
172,11691.0,5952,11691,11691,67094,MULTIPOLYGON (((-73.78915889414289 40.59752681...,POINT (-73.76135910415132 40.60132941338052),-73.761359,40.601329
173,11692.0,1672,11692,11692,20991,MULTIPOLYGON (((-73.80276329921944 40.59261580...,POINT (-73.79219583977951 40.5927973858467),-73.792196,40.592797
174,11693.0,1372,11693,11693,12919,MULTIPOLYGON (((-73.82010481980066 40.58699545...,POINT (-73.81671090698718 40.596457309804165),-73.816711,40.596457
175,11694.0,1798,11694,11694,21354,MULTIPOLYGON (((-73.86495602260939 40.56663006...,POINT (-73.84455125213255 40.57759893303544),-73.844551,40.577599
